In [3]:
import cv2
import numpy as np
import os


from keras.models import load_model
model = load_model('C:\\Users\\prabh\\Sign-Language-to-Text-Conversion1\\model_save')

In [5]:
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    # Simulating mirror image
    frame = cv2.flip(frame, 1)
    
    x1 = int(0.7*frame.shape[1])
    y1 = 100
    x2 = frame.shape[1]-10
    y2 = int(0.45*frame.shape[1])
    # Drawing the ROI
    # The increment/decrement by 1 is to compensate for the bounding box
    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
    # Extracting the ROI
    roi = frame[y1:y2, x1:x2]
    roi = cv2.resize(roi, (100, 100)) 
    
    
    #_, mask = cv2.threshold(mask, 200, 255, cv2.THRESH_BINARY)
    #kernel = np.ones((1, 1), np.uint8)
    #img = cv2.dilate(mask, kernel, iterations=1)
    #img = cv2.erode(mask, kernel, iterations=1)
    # do the processing after capturing the image!
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    #_, roi = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
    roi = cv2.Canny(roi,80,80)
    
    roi1 = roi.copy()
    cv2.imshow("ROI", roi)
    
    
    roi1.resize(1,100,100,1)
    classIndex = int(np.argmax((model.predict(roi1))))
    
    m = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'nothing',15:'O',16:'P',17:'Q',18:'R',19:'S',20:'T',21:'U',22:'V',23:'W',24:'X',25:'Y',26:'Z'}
    
    cv2.putText(frame, "Alphabet : "+str(m[classIndex]), (10, 430), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 2)
    
    cv2.imshow("Frame", frame)
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27: # esc key
        break
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [1]:
import cv2
import numpy as np
from keras.models import load_model
import time
import tkinter as tk
from PIL import Image, ImageTk
from threading import Thread

# Load the trained model
model_file = './model_save'
try:
    model = load_model(model_file)
except FileNotFoundError:
    print(f"Model file '{model_file}' not found.")
    exit()

# Mapping index to corresponding letter or "nothing"
letters_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
                   10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'nothing', 15: 'O', 16: 'P', 17: 'Q', 18: 'R',
                   19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z'}

# Create a class for the GUI
class SignLanguageGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sign Language Recognition")

        self.video_label = tk.Label(root)
        self.video_label.pack()

        self.letter_box_label = tk.Label(root, text="Current Letter:")
        self.letter_box_label.pack()

        self.letter_var = tk.StringVar()
        self.letter_var.set("")
        self.letter_box = tk.Label(root, textvariable=self.letter_var, font=("Helvetica", 16))
        self.letter_box.pack()

        self.word_label = tk.Label(root, text="Formed Word:")
        self.word_label.pack()

        self.word_var = tk.StringVar()
        self.word_var.set("")
        self.word_display = tk.Label(root, textvariable=self.word_var, font=("Helvetica", 16))
        self.word_display.pack()

        self.sentence_label = tk.Label(root, text="Formed Sentence:")
        self.sentence_label.pack()

        self.sentence_var = tk.StringVar()
        self.sentence_var.set("")
        self.sentence_display = tk.Label(root, textvariable=self.sentence_var, font=("Helvetica", 16))
        self.sentence_display.pack()

        self.last_recognized_time = time.time()
        self.letter = ""
        self.word = ""
        self.sentence = ""

        self.camera = cv2.VideoCapture(0)

        # Start video processing in a separate thread
        self.video_thread = Thread(target=self.show_video)
        self.video_thread.daemon = True
        self.video_thread.start()

        # Add "Next," "Enter," and "Close" buttons
        next_button = tk.Button(root, text="Next", command=self.add_letter_to_word)
        next_button.pack()

        enter_button = tk.Button(root, text="Enter", command=self.add_word_to_sentence)
        enter_button.pack()

        close_button = tk.Button(root, text="Close", command=self.close_program)
        close_button.pack()

    def show_video(self):
        while True:
            _, frame = self.camera.read()
            frame = cv2.flip(frame, 1)

            # Create a separate camera box (ROI) for sign language input
            roi = frame[100:300, 400:600]

            # Draw a rectangle (visible square) around the ROI
            cv2.rectangle(frame, (400, 100), (600, 300), (0, 255, 0), 2)

            # Preprocess the ROI (similar to the preprocessing done during training)
            roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            roi_canny = cv2.Canny(roi_gray, 80, 80)
            roi_resized = cv2.resize(roi_canny, (100, 100))
            roi_normalized = roi_resized.reshape(1, 100, 100, 1) / 255.0

            # Make a prediction using the model
            class_probabilities = model.predict(roi_normalized)
            class_index = np.argmax(class_probabilities)

            # Get the predicted letter
            predicted_letter = letters_mapping[class_index]

            # If the predicted letter is 'nothing', add a space to the letter box
            if predicted_letter == "nothing" and time.time() - self.last_recognized_time > 1.5:
                self.letter = ' '
                self.last_recognized_time = time.time()

            # If the predicted letter is not 'nothing' and a certain time has passed since the last recognition
            elif predicted_letter != "nothing" and time.time() - self.last_recognized_time > 1.5:
                if predicted_letter == 'F':  # Use 'F' as a gesture for space
                    self.letter = ' '
                else:
                    self.letter = predicted_letter
                self.last_recognized_time = time.time()

            # Display the video frame
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            img = ImageTk.PhotoImage(image=img)
            self.video_label.img = img
            self.video_label.config(image=img)
            self.video_label.update()

    def add_letter_to_word(self):
        # Add the current letter to the word
        self.word += self.letter
        self.letter = ""
        self.letter_var.set("Current Letter: " + self.letter)
        self.word_var.set("Formed Word: " + self.word)

    def add_word_to_sentence(self):
        # Add the current word to the sentence
        self.sentence += self.word + ' '
        self.word = ""
        self.word_var.set("Formed Word: " + self.word)
        self.sentence_var.set("Formed Sentence: " + self.sentence)

    def close_program(self):
        # Release the camera when the GUI is closed
        self.camera.release()
        cv2.destroyAllWindows()
        self.root.destroy()

# Create the main window and run the GUI
root = tk.Tk()
app = SignLanguageGUI(root)
root.mainloop()


1/1 [==============================] - 0s 41ms/step
